## 4.1.	Violações dos padrões de qualidade ar no Brasil



### 4.1.1	Séries temporais das estações de monitoramento


**TENTAR FAZER UM DROPDOWN MENU PARA ESCOLHER QUAL SÉRIE TEMPORAL VISUALIZAR NO LIVRO. Colocar padrões da CONAMA**

ou Figura 5 – Perfil temporal das médias diárias de a) MP10, b) MP2,5 e c) O3 nas estações Volta da Capela (Barra Longa – MG) e Vila Moema (Tubarão – SC) no ano de 2023.


### 4.1.2	Estatísticas das estações de monitoramento 


**TENTAR FAZER UMA TABELA ITERATIVA ONDE O USUÁRIO POSSA FILTRAR AS ESTAÇÕES E NÚMERO DE VIOLAÇÕES**  
ou Tabela 10 – Ranqueamento das cinco estações com o maior número de violações dos Padrões de Qualidade do Ar de O 3, MP10, MP2,5 e SO2 no ano de 2023.

```{note}
Here is a note
```

```{warning}
Here is a warning
```

```{important}
Here is a important
```

```{caution}
Here is a caution
```

```{tip}
Here is a tip
```

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

rootPath = os.path.dirname(os.getcwd())
data_folder = rootPath + '/data/MQAr'

# List CSV files
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# Widgets
file_dropdown = widgets.Dropdown(
    options=csv_files,
    description='CSV:',
)

column_dropdown = widgets.Dropdown(
    options=['VALOR'],
    description='Column:'
)

date_column_input = widgets.Text(
    value='date',
    description='Date col:',
    placeholder='Enter date column name'
)

output = widgets.Output()

def update_column_dropdown(change):
    try:
        file_path = os.path.join(data_folder, file_dropdown.value)
        df = pd.read_csv(file_path, encoding='utf-8')
        column_dropdown.options = [col for col in df.columns if col != date_column_input.value]
    except Exception as e:
        column_dropdown.options = []
        with output:
            clear_output()
            print(f"Error reading file: {e}")

def plot_csv(file, column, date_col):
    with output:
        clear_output()
        try:
            file_path = os.path.join(data_folder, file)
            df = pd.read_csv(file_path, encoding='utf-8')

            df[date_col] = pd.to_datetime({
                'year': df['ANO'],
                'month': df['MES'],
                'day': df['DIA'],
                'hour': pd.to_datetime(df['HORA'], format='%H:%M:%S').dt.hour
            })

            df = df.dropna(subset=[date_col, column])
            df = df.sort_values(date_col)

            plt.figure(figsize=(10, 4))
            plt.plot(df[date_col], df[column], marker='o', linestyle='-')
            plt.xlabel(date_col)
            plt.ylabel(column)
            plt.title(f"{column} over time")
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Error plotting: {e}")

file_dropdown.observe(update_column_dropdown, names='value')

plot_button = widgets.Button(description="Plot")
plot_button.on_click(lambda b: plot_csv(file_dropdown.value, column_dropdown.value, date_column_input.value))

ui = widgets.VBox([file_dropdown, date_column_input, column_dropdown, plot_button, output])
display(ui)

